# Data preparation

Below are the steps involved to understand, clean and prepare your data for building your predictive model:

   1.  Variable Identification
   2.  Missing values treatment
   3.  Outlier treatment
   4.  Encoding
   5.  A little Viz
   
Finally, we will need to iterate over steps 4 – 7 multiple times before we come up with our refined model.

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Minmax scaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
house=pd.read_csv("train.csv")

# Variable Identification 

Identify Predictor (Input) and Target (output) variables 

For example for the given dataset, problem statements could be, 

1. Given the information of the patient what is 

In [ ]:
house.info()

 Seperate them into Datatypes of catogarical and Continuouse ones 

In [ ]:
a = house.dtypes[house.dtypes == 'object'].index
house[a].head()

# Handling Missing Values
We treat missing values in a variety of ways, 

    1. Deletion
    2. Mean/Mode/Median imputation
    3. Prediction Model: In this we train a model that is let to overfit to predict the missing values. 
    
What we use is subjective and depends on the use case. 

In [ ]:
house.Street.value_counts()

In [ ]:
house.Alley.value_counts()

In [ ]:
house.Alley.isna().sum()

In [ ]:
house.drop('Alley',inplace=True, axis=1)

In [ ]:
a = house.dtypes[house.dtypes == 'object'].index
house[a].head()

In [ ]:
house[a].isna().sum()

In [ ]:
house.BsmtCond.value_counts()

In [ ]:
b = house.dtypes[house.dtypes == 'float64'].index
house[b].isnull().sum()

In [ ]:
house.MasVnrArea.fillna(0,inplace=True)

In [ ]:
house.GarageYrBlt.value_counts()

In [ ]:
c = house.dtypes[house.dtypes == 'int64'].index
house[c].isnull().sum()

In [ ]:
house[a].columns

In [ ]:
house[['MasVnrType', 'BsmtQual', 'BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2', 'FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MiscFeature']]

As you can see PoolQC,Fence,MiscFeatures for example have many nan values but they definitely add a lot of value in determining the house prices and hence removing the column doesn't make sense. It can safely be said that the NaN values here maybe a No. So we will use a fillna('No')

Check out ffill and bfill methods in fillna. They are usually used in timeseries/order sensitive data where missing values could be easily identified and replaced based on trends.

In [ ]:
house.fillna({'MasVnrType':'No', 'BsmtQual':'No', 'BsmtCond':'No','BsmtExposure':'No','BsmtFinType1':'No','BsmtFinType2':'No', 'FireplaceQu':'No','GarageType':'No','GarageFinish':'No','GarageQual':'No','GarageCond':'No','PoolQC':'No','Fence':'No','MiscFeature':'No'}, inplace=True)
house

In [ ]:
house.Electrical.value_counts()
#Replace with mode
house['Electrical'].fillna("SBrkr",inplace=True)

For example, LotFrontage has a lot of null values, around 259 of em. The way we go about filling these values can go in various directions. Let's see. The null values might be because they are essentially not attached to any street inherently, so it might make sense to replace them with 0. But, 

In [ ]:
house[house.LotFrontage.isnull()].LotConfig.value_counts()

This means that that might not be the case. So we need to fill in some sensible value here. One direction of thinking might be that houses in a specific neighborhood will have similar LofFrontage due to their arrangement, so we substitute the nulls values with the mean of LotFrontage of various neighbourhoods. 

In [ ]:
house[['Neighborhood', 'LotFrontage']].groupby(['Neighborhood']).transform(lambda x: x.fillna(x.mean()))

In [ ]:
house[['Neighborhood', 'LotFrontage', 'LotConfig']].groupby(['Neighborhood', 'LotConfig']).transform(lambda x: x.fillna(x.mean()))

What is better? Well, the latter seems more accurate but you'll know if your approach makes sense once you train and see the results. 

# Skew Removal

In [ ]:
plt.hist(house.LotArea, bins = 100)

In [ ]:
plt.hist(np.log(house.LotArea), bins = 100)


Kurtosis is a measure of tailed-ness or spread of the distribution. mathematically it is calculated using the expectation of the 4 degree of variation from the mean, which would be E((X - E[X])^4), divided by the 4th power of standard deviation.

In [ ]:
house.LotArea.kurtosis()

This tailed-ness needs to be removed, as the tailed-ness is due to outliers. We can use the box plot concept to identify the outliers, and then replace them with mean/median, or use winsorisation (use the extremum values)

In [ ]:
def remove_outliers(x):
    q1 = house.LotArea.quantile(q = 0.25)
    q3 = house.LotArea.quantile(q = 0.75)
    iqr = q3 - q1
    outlier_range = 1.5*iqr
    r_whisker = q3 + outlier_range
    l_whisker = q1 - outlier_range
    if (x > r_whisker):
        return q3
    elif (x < l_whisker):
        return q1
    else: 
        return x
    
house.LotArea.apply(lambda x: remove_outliers(x)).kurtosis()

This has decreased the kurtosis a lot, but we are loosing a lot of original values. So it might be best to test both cases. 

# Categorical Values

Sometimes, the numbers might not make sense as numbers. Sometimes the columns won't have numbers at all. We need to handle them well as your model can only understand numbers.

Categories mainly come as two types, ordinal and non-ordinal. Ordinal values simply have a inherent order in the categories, like "low, medium, high", years and months. Non-ordinal would be values which have no inherent order in them like countries. 

Some values might might make more sense as binned categories than their intrinsic numerical values. For example time. Years and months are represented as numbers but they aren't numbers, so it makes more sense to label encode them. 

In [ ]:
house.YrSold = house.YrSold.astype(str)
house.MoSold = house.MoSold.astype(str)

Some numerical values are very discreet, and it makes more sense to bin them into respective categories, like MSSubClass which is the class of the building and the value here is some type of a code than a number. 

In [ ]:
house.MSSubClass.value_counts()
# house.MSSubClass = house.MSSubClass.astype(str)

### In order to treat them as categories, we need to either label encode them or one-hot encode them. 

Label-encoding simply gives them values like 0,1,2.... whereas one-hot encoding takes each category as a column and throw a value 1 for positive hit and 0 for a negative hit. There is an issue with label-encoding, as you are giving orderly values to categorical data which might not even be ordered, effectively changing the data and the model's understanding of the data. In such case we use the one-hot encoding method, but even this has an issue of multi-collinearity, meaning correlation between various columns. How? Well, if you find a 1 in one of the columns, it would mean that the other columns will definitely be 0. 

In [ ]:
#label encoding
lencoder = LabelEncoder() 
lencoder.fit_transform(house.MSSubClass) 

In [ ]:
house.MSSubClass

This doesn't make sense as the column isn't actually ordered. So we'll use one-hot encoding. 

In [ ]:
dumm = pd.get_dummies(house.MSSubClass)
# pd.concat([house, dumm], axis = 1)
# house.drop(columns=[['MSSubClass']])

Now how do you ensure multicollinearity isn't a problem? We have a statistical measure that we can check. It's called variance inflation factor (VIF). Due to time constraint, we recommend you read [this](https://online.stat.psu.edu/stat462/node/180/) for VIF. Put your queries on slack if you have any doubts.   

In [ ]:
def calculate_vif(data):
    vif_df = pd.DataFrame(columns = ['Var', 'Vif'])
    x_var_names = data.columns
    for i in range(0, x_var_names.shape[0]):
        y = data[x_var_names[i]]
        x = data[x_var_names.drop([x_var_names[i]])]
        r_squared = sm.OLS(y,x).fit().rsquared
        vif = round(1/(1-r_squared),2)
        vif_df.loc[i] = [x_var_names[i], vif]
    return vif_df.sort_values(by = 'Vif', axis = 0, ascending=False, inplace=False)

vifcalc = pd.get_dummies(house.MSSubClass)
calculate_vif(vifcalc)

What do we do now? we drop the dummy columns which have very high VIF (above 5). After that, recalculate the VIF for all columns and you'll see a drop. 

What do we use? There are some heuristics you can follow, 
1. Use one-hot encoding if the number of categories are low, so you don't end up overloading your data and increasing memory required for training, else use label encoding. 
2. Use label encoding if the data is inherently ordered (ordinal categories). 

# visualising Categorical values

1. A simple bar plot will give you a really good picture of what's happening. If you want to visualise on a condition, like a groupby, you can use grouped bar plots.

In [ ]:
plt.figure(figsize=(20,10))
sns.countplot(house.Neighborhood)

In [ ]:
plt.figure(figsize=(30,10))
sns.countplot(data=house, x= 'Neighborhood', hue = 'LotConfig')